In [5]:
import pandas as pd

## Création des constantes et des quelques données nécessaires aux traitements

In [6]:
import os
path_to_csv = "C:/Users/maila/Documents/Centrale Nantes/EI2/INFOIA/STASC/Data_Challenge/_classified/train"

# On récup la liste de l'ensemble des fichiers .csv créées à l'aide du modèle de DL RoomNet que j'ai adapté ici
list_of_files = os.listdir(path_to_csv)

CLASS = ['Backyard', 'Bathroom', 'Bedroom', 'Frontyard', 'Kitchen', 'LivingRoom']
POND = [2, 0.5, 1, 1.5, 1.5, 2]


## Traitement des .xls pour obtenir un unique Dataframe avec un score en fonction des images pour chaque annonce

In [7]:
# création du dictionnaire final, qui contiendra toutes les données des excels et les scores (sans les valeurs de fiabilité)
donnees_finales = pd.DataFrame({'id_annonce':[],'Backyard' : [], 'Bathroom' : [], 'Bedroom' : [], 'Frontyard' : [],'Kitchen' : [], 'LivingRoom' : [],'score':[]})
dict_operations = {'Value_Count': ('PREDICTED_LABEL', 'count'), 'average_fiability': ('Unnamed: 2', 'mean')} # dictionnaire qui contient les opérations à effectuer sur les colonnes des dataframes qu'on récup

i=0
for file in list_of_files:
    if i%1000 == 0: # compteur pour suivre l'avancement du traitement
        print(i)
    i = i+1
    A = pd.read_excel(path_to_csv + "/" + file)
    elem_presents = A.groupby('PREDICTED_LABEL', as_index=False).agg(**dict_operations) # on crée un dataframe avec le count des différentes classes et la fiabilité moyenne par classe

    dict_temp = {} # création du dictionnaire temporaire qui contiendra la valeurs avant de les mettre dans le dataframe final
    score = 0 # initialisation du score
    for elem in elem_presents['PREDICTED_LABEL']:
        number_of_elem = elem_presents['Value_Count'][elem_presents['PREDICTED_LABEL'] == elem].tolist()[0] # on récupère le nombre d'éléments de la classe
        average_fiability = elem_presents['average_fiability'][elem_presents['PREDICTED_LABEL'] == elem].tolist()[0] # on récupère la fiabilité moyenne de la classe
        dict_temp[elem] = [number_of_elem] # on met le nombre d'éléments de la classe dans le dictionnaire
        score = score + number_of_elem*POND[CLASS.index(elem)]*average_fiability # on calcule le score en multipliant notre pondération et la fiabilité moyenne de la classe

    score = [score]
    dict_temp['score'] = score # on ajoute le score au dictionnaire
    dict_temp['id_annonce'] = file[4:-12] # on ajoute l'id de l'annonce au dictionnaire

    temp = pd.DataFrame(dict_temp)
    donnees_finales = pd.concat([donnees_finales,temp],ignore_index=True)


0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [8]:
donnees_finales.to_csv('donnees_finales_images_train.csv', index=False)

In [9]:
image_test = pd.read_csv('donnees_finales_images_test.csv')
image_train = pd.read_csv('donnees_finales_images_train.csv')

In [10]:
# Concaténation de test et train
image = pd.concat([image_test,image_train],ignore_index=True)

In [11]:
X = pd.read_csv('../Prix_metre/X_avec_prix_complet2.csv')

In [12]:
# Merge X et image sur l'id_annonce
X = pd.merge(X, image, on='id_annonce', how='left')

In [13]:
X.to_csv('X_avec_prix_et_images.csv', index=False)